In [26]:
import logging
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# Importar Módulos Langchain
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import LlamaCpp, CTransformers

# Importar módulo para Front-End Web
import gradio as gr

# Importar módulo para carregar variáveis de ambiente do projeto
from dotenv import load_dotenv
import os

In [27]:
class ChatPDF:

    def __init__(self, documents_folder: str, vectordb_folder: str, model_path: str, sentence_embedding_model: str, temperature: float = 0.1):
        """
        Constrói todos os atributos necessários para o objeto ChatPDF

        Args:
            documents_folder (str): Caminho/Pasta com os documentos que serão carregados para conversação no Chat
            vectordb_folder (str): Caminho/Pasta onde ficarão os arquivos do Chroma (Vector DB)
            model_path (str): Caminho/Pasta que aponta para o modelo LLM a ser utilizado
            sentence_embedding_model (str): Nome do modelo de Embedding que será usado para gerar os tokens dos documentos
            temperature (float, optional): Temperatura para calibrar o nível de aleatoriedade das respostas. O padrão é 0.1 (Muito determinístico, pouco aleatório)
        """
        self.documents_folder = documents_folder
        self.vectordb_folder = vectordb_folder
        self.model_path = model_path
        self.sentence_embedding_model = sentence_embedding_model
        self.temperature = temperature
        self.pages = []
        self.chunks = []

    def load(self) -> int:
        """
        Realiza a carga dos documentos do caminho/pasta definido no atributo documents_folder.

        Returns:
            int: Quantidade total de páginas carregadas de todos os arquivos PDF
        """

        loader = DirectoryLoader(
            self.documents_folder,
            glob="**/*.pdf",
            loader_cls=PyMuPDFLoader,
            show_progress=True,
            use_multithreading=True
        )

        self.pages = loader.load()

        return len(self.pages)
    
    def split(self, chunk_size: int = 800, chunk_overlap: int = 400) -> int:
        """
        Realiza o split das páginas em chunks para armazenar no Vector DB

        Args:
            chunk_size (int, optional): Quantidade máxima de caracteres de cada chunk. O padrão é 1500.
            chunk_overlap (int, optional): Quantidade de caracteres de overlap entre chunks. O padrão é 150.

        Returns:
            int: Quantidade total de chunks de todas as páginas de todos os documentos carregados
        """

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )

        self.chunks = text_splitter.split_documents(self.pages)

        return len(self.chunks)
    
    def get_embeddings(self):
        """
        Obtem os embeddings do modelo de linguagem definido no atributo sentence_embedding_model.
        """
        self.embeddings = SentenceTransformerEmbeddings(model_name=self.sentence_embedding_model)

    def store(self):
        """
        Armazena os chunks de todos os documentos no Vector DB, utilizando o embedding definido. 
        """
        vectordb = Chroma.from_documents(
            documents=self.chunks,
            embedding=self.embeddings,
            persist_directory=self.vectordb_folder
        )

        vectordb.persist()

        self.vectordb = vectordb

    def create_llm(self):
        """
        Cria uma LLM local, com base no modelo definido no atributo model_path.
        """
        
        self.llm = LlamaCpp(model_path=self.model_path, verbose=True, n_ctx=2048, temperature=self.temperature)

    def create_retriever(self):
        """
        Cria um retriever de documentos com base do Vector DB já carregado com os documentos.
        """
        self.retriever = self.vectordb.as_retriever(search_kwargs={'k': 3})

    def create_qa_session(self):
        """
        Cria uma sessão de QA, usando o LLM e Retriever já instanciados.
        """

        PROMPT_TEMPLATE = """
            Você é um assistente virtual, que responde exclusivamente em português.
            Use o contexto fornecido para responder à pergunta de forma clara e concisa.
            Se e pergunta for fora do contexto dos PDF, responda que não pode responder fora do tópico.
            Exemplo de pergunta fora do contexto: O que é arroz?
            Exemplo de resposta para perguntas fora do contexto: Não posso responder perguntas fora do tópico.
            Contexto: {context}
            Pergunta: {question}
            Resposta:"""

        QA_CHAIN_PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)

        self.qa = RetrievalQA.from_chain_type(
            self.llm,
            'stuff',
            retriever=self.retriever,
            return_source_documents=True,
            chain_type_kwargs={'prompt': QA_CHAIN_PROMPT}
        )

In [28]:
# Carrega as variáveis de ambiente do arquivo .env, para utilizar na hora de parametrizar o objeto ChatPDF
load_dotenv()

MODEL_PATH = os.getenv('MODEL_PATH')
VECTORDB_FOLDER = os.getenv('VECTORDB_FOLDER')
DOCUMENTS_FOLDER = os.getenv('DOCUMENTS_FOLDER')
SENTENCE_EMBEDDING_MODEL = os.getenv('SENTENCE_EMBEDDING_MODEL')

In [29]:
# Cria um objeto ChatPDF
chat = ChatPDF(DOCUMENTS_FOLDER, VECTORDB_FOLDER, MODEL_PATH, SENTENCE_EMBEDDING_MODEL, temperature=0.1)

In [30]:
# Executa a carga dos documentos PDF que iremos interagir na sessão de Chat Q&A
chat.load()

100%|██████████| 2/2 [00:00<00:00, 50.00it/s]


29

In [31]:
# Executa o split dos documentos carregados
chat.split()

33

In [32]:
# Obtem os embeddings do modelo de linguagem selecionado
chat.get_embeddings()

In [33]:
# Armazena os chunks dos documentos, junto com o embedding, no Vector DB (Chroma)
chat.store()

In [34]:
# Cria a LLM (LLAMA V2) localmente para interagirmos na sessão de chat
chat.create_llm()

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from D:\Workspace Irede\llamav2_local\models\llama3 (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-8B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.

In [35]:
# Cria o retriever, que irá recuperar os documentos do Vector DB, com base nos Prompts e usando a LLM
chat.create_retriever()

In [36]:
# Cria uma sessão de chat para Q&A, com base no LLM e Retriever
chat.create_qa_session()

In [37]:
# Front-End da Aplicação Web - Gradio

chat_history = []

with gr.Blocks() as demo:

    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    chat_history = []
    
    def user(user_message, chat_history):
        
        # Retorna resposta da LLM, através da sessão de Q&A
        result = chat.qa({"query": user_message})
        
        # Realiza um append na tela do chat, contendo a mensagem do usuário e a resposta do modelo
        chat_history.append((user_message, result["result"]))

        return gr.update(value=""), chat_history
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)

if __name__ == "__main__":
    demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.



llama_print_timings:        load time =    1584.98 ms
llama_print_timings:      sample time =     139.03 ms /   256 runs   (    0.54 ms per token,  1841.33 tokens per second)
llama_print_timings: prompt eval time =   30971.63 ms /   311 tokens (   99.59 ms per token,    10.04 tokens per second)
llama_print_timings:        eval time =   48360.93 ms /   255 runs   (  189.65 ms per token,     5.27 tokens per second)
llama_print_timings:       total time =   79959.36 ms /   566 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1584.98 ms
llama_print_timings:      sample time =     158.63 ms /   256 runs   (    0.62 ms per token,  1613.79 tokens per second)
llama_print_timings: prompt eval time =   22875.81 ms /   194 tokens (  117.92 ms per token,     8.48 tokens per second)
llama_print_timings:        eval time =   59437.34 ms /   255 runs   (  233.09 ms per token,     4.29 tokens per second)
llama_print_timings:       total time =   82998.96 ms /   449 

Keyboard interruption in main thread... closing server.
